In [ ]:
# Instala la herramienta Consola del archivo ISIC
!pip install isic-cli #Sacada desde el repositorio de Github https://github.com/ImageMarkup/isic-cli/blob/master/README.md

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00


In [ ]:
#Testeado y funcionando
!isic image download --search "" --collections "65" --limit 0 Training2019/ # Imagenes y Metadatos de Entrenamiento Año 2019

If you have been granted special permissions, logging in with `isic user login` might return more data.


Successfully downloaded 25,331 images to Training2019/.
Successfully wrote 25,331 metadata records to Training2019/metadata.csv.
Successfully wrote attributions to Training2019/attribution.txt.
Successfully wrote 2 license(s) to Training2019/licenses.


-------------------------------------------------------------------------

In [ ]:
import csv

def cargar_metadatos(archivo):
    """
    Carga los metadatos desde un archivo CSV y filtra las imágenes con diagnóstico Benign (0) y Malignant (1).
    Retorna un diccionario con isic_id y las etiquetas mapeadas a 0 y 1.
    """
    datos = {
        'isic_id': [],
        'diagnosis_1': []  # Las etiquetas serán 0 o 1
    }
    etiquetas_validas = {'Benign': 0, 'Malignant': 1}  # Mapeamos las etiquetas a 0 y 1

    try:
        with open(archivo, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader)  # Evita las columnas cabeceras.

            diagnosis_idx =9  # Cambiar al índice correspondiente si es necesario
            #7 para BCN20000
            # 9 para training2019
            for fila in reader:
                if len(fila) > diagnosis_idx:
                    diagnosis = fila[diagnosis_idx]
                    if diagnosis in etiquetas_validas:  # Solo almacenar Benign y Malignant
                        datos['isic_id'].append(fila[0])  # Almacena el ID de la imagen
                        datos['diagnosis_1'].append(etiquetas_validas[diagnosis])  # Asignamos la etiqueta 0 o 1

        print(f"Datos extraídos y filtrados correctamente. Se encontraron {len(datos['isic_id'])} registros válidos.")
        return datos

    except FileNotFoundError:
        print(f"Error: Archivo {archivo} no encontrado")
        return None

    except Exception as e:
        print(f"Error al procesar el archivo: {e}")
        return None

In [ ]:
import os
import shutil
# Se elimina: from sklearn.model_selection import train_test_split
from tqdm import tqdm

def organizar_imagenes_en_carpetas(carpeta_origen, metadatos, carpeta_destino_base):
    """
    Organiza el 100% de las imágenes en carpetas 'test' según su clase.
    """
    # --- 1. Crear las carpetas de destino ---
    # El loop solo crea el directorio 'test'
    for split in ['test']:
        for class_name in ['benign', 'malignant']:
            os.makedirs(os.path.join(carpeta_destino_base, split, class_name), exist_ok=True)

    # --- 2. Separar los IDs por clase ---
    ids_benignos = []
    ids_malignos = []
    for id_img, etiqueta in zip(metadatos['isic_id'], metadatos['diagnosis_1']):
        if etiqueta == 0:  # Benign
            ids_benignos.append(id_img)
        elif etiqueta == 1: # Malignant
            ids_malignos.append(id_img)

    # --- 3. Asignar el 100% de los IDs al conjunto de 'test' ---
    # [Lógica: Modificación solicitada]
    # No se usa train_test_split para enviar todo a 'test'.
    # Se asigna la lista completa de IDs de cada clase.
    ids_benignos_test = ids_benignos
    ids_malignos_test = ids_malignos

    # --- 4. Función auxiliar para copiar archivos ---
    def copiar_archivos(ids, carpeta_destino):
        for id_img in tqdm(ids, desc=f'Copiando a {os.path.basename(os.path.normpath(carpeta_destino))}'):
            nombre_archivo = f"{id_img}.jpg"
            ruta_origen = os.path.join(carpeta_origen, nombre_archivo)
            ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
            if os.path.exists(ruta_origen):
                shutil.copy(ruta_origen, ruta_destino)
            else:
                # [Mejor Práctica]: Es preferible usar 'warning' o 'logging'
                print(f"Advertencia: No se encontró el archivo {ruta_origen}")

    # --- 5. Copiar los archivos a sus directorios finales ---

    print("Copiando imágenes de prueba benignas...")
    copiar_archivos(ids_benignos_test, os.path.join(carpeta_destino_base, 'test', 'benign'))

    print("Copiando imágenes de prueba malignas...")
    copiar_archivos(ids_malignos_test, os.path.join(carpeta_destino_base, 'test', 'malignant'))

    # Se actualiza el mensaje final para reflejar la acción
    print(f"\nEl 100% de las imágenes ({len(ids_benignos_test) + len(ids_malignos_test)} archivos) se ha organizado correctamente en la carpeta 'test'.")

In [ ]:
import os
import shutil

# Eliminar el directorio del dataset antiguo para evitar conflictos y asegurar una regeneración limpia
if os.path.exists('/content/dataset'):
    shutil.rmtree('/content/dataset')
    print("Directorio '/content/dataset' anterior eliminado.")
else:
    print("El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.")

El directorio '/content/dataset' no existía, no fue necesaria ninguna limpieza.


In [ ]:
# Asumiendo que los metadatos ya han sido cargados correctamente
metadatos = cargar_metadatos("/content/Training2019/metadata.csv")

Datos extraídos y filtrados correctamente. Se encontraron 24463 registros válidos.


In [ ]:
organizar_imagenes_en_carpetas('/content/Training2019/', metadatos, '/content/dataset')

Copiando imágenes de prueba benignas...


Copiando a benign: 100%|██████████| 15990/15990 [00:10<00:00, 1565.97it/s]


Copiando imágenes de prueba malignas...


Copiando a malignant: 100%|██████████| 8473/8473 [00:06<00:00, 1231.89it/s]


El 100% de las imágenes (24463 archivos) se ha organizado correctamente en la carpeta 'test'.


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def crear_generador_para_inferencia(carpeta_test, batch_size=32, target_size=(224, 224)):

    # Definición del Generador
    # que se usó en el entrenamiento (en este caso, el rescalado).
    test_datagen = ImageDataGenerator(rescale=1./255)


    # mantengan el orden de los archivos.
    inference_generator = test_datagen.flow_from_directory(
        carpeta_test,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='sparse',
        shuffle=False
    )

    print(f"Generador de inferencia creado. Apuntando a: {carpeta_test}")
    print(f"Imágenes encontradas: {inference_generator.n}")

    # Se retorna tanto la configuración (datagen) como el iterador (flow)
    return test_datagen, inference_generator

RUTA_TEST = '/content/dataset/test'

# batch_size=1 para predicciones individuales
datagen_inferencia, generador_inferencia = crear_generador_para_inferencia(
    RUTA_TEST,
    batch_size=1
)

Found 24463 images belonging to 2 classes.
Generador de inferencia creado. Apuntando a: /content/dataset/test
Imágenes encontradas: 24463


In [ ]:
!pip install tf_keras tensorflow_hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tf_keras
import numpy as np

# Rutas
MODEL_PATH = "/content/model_v1.h5"

print(f"Cargando modelo legacy desde {MODEL_PATH}...")

try:
    # Definir objetos personalizados
    custom_objects_dict = {
        'KerasLayer': hub.KerasLayer,
        'TrueDivide': tf.math.truediv
    }

    model = tf_keras.models.load_model(MODEL_PATH, custom_objects=custom_objects_dict)

    print("Modelo cargado exitosamente con tf_keras.")

except Exception as e:
    print(f"Error fatal al cargar: {e}")
    raise

# Realizando Inferencia
print("Realizando predicciones (inferencia)...")

y_pred_proba = model.predict(
    generador_inferencia,
    steps=generador_inferencia.n,
    verbose=1
)


print(f"Inferencia completada. Se obtuvieron {len(y_pred_proba)} predicciones.")
print("\n--- Celda 2 Ejecutada ---")

Cargando modelo legacy desde /content/model_v1.h5...
✅ Modelo cargado exitosamente con tf_keras.
Realizando predicciones (inferencia)...
24463/24463 [==============================] - 2348s 96ms/step
Inferencia completada. Se obtuvieron 24463 predicciones.

--- Celda 2 Ejecutada ---


In [ ]:
import numpy as np
import pandas as pd
import os

CSV_OUTPUT_PATH = "resultados_validacion_ModeloTL.csv"
EXCEL_OUTPUT_PATH = "resultados_validacion_ModeloTL.xlsx"


# Procesar Predicciones y Datos Reales
print("Procesando resultados de la inferencia...")

# Convertir probabilidades a etiquetas (0 o 1)
# np.argmax encuentra el índice (0 o 1) con la probabilidad más alta
y_pred_labels = np.argmax(y_pred_proba, axis=1)

# Obtener la probabilidad (confianza) de la predicción
y_pred_scores = np.max(y_pred_proba, axis=1)

y_true_labels = generador_inferencia.classes
filenames = generador_inferencia.filenames

# Ensamble del DataFrame
print("Ensamblando DataFrame...")

# Crear un mapa para convertir 0 -> 'benign', 1 -> 'malignant'
label_map = {v: k for k, v in generador_inferencia.class_indices.items()}

df = pd.DataFrame({
    'nombre_imagen': [os.path.basename(f) for f in filenames],
    'diagnostico_original_id': y_true_labels,
    'diagnostico_propuesto_id': y_pred_labels,
    'confianza_prediccion': y_pred_scores
})

# Mapear los IDs a nombres legibles
df['diagnostico_original'] = df['diagnostico_original_id'].map(label_map)
df['diagnostico_propuesto'] = df['diagnostico_propuesto_id'].map(label_map)

# Seleccionar y ordenar las columnas finales
df_final = df[[
    'nombre_imagen',
    'diagnostico_original',
    'diagnostico_propuesto',
    'confianza_prediccion'
]]

# Guardar en Archivos
try:
    # Guardar en CSV
    df_final.to_csv(CSV_OUTPUT_PATH, index=False, encoding='utf-8')
    print(f"\n¡Éxito! Resultados guardados en {CSV_OUTPUT_PATH}")

    # Guardar en Excel
    df_final.to_excel(EXCEL_OUTPUT_PATH, index=False)
    print(f"¡Éxito! Resultados convertidos a Excel y guardados en {EXCEL_OUTPUT_PATH}")

    # Mostrar las primeras 5 filas
    print("\nPrimeras 5 filas del resultado:")
    print(df_final.head())

except Exception as e:
    print(f"Error al guardar los archivos de resultados: {e}")

print("\n--- Celda 3 Ejecutada ---")

Procesando resultados de la inferencia...
Ensamblando DataFrame...

¡Éxito! Resultados guardados en resultados_validacion_ModeloTL.csv
¡Éxito! Resultados convertidos a Excel y guardados en resultados_validacion_ModeloTL.xlsx

Primeras 5 filas del resultado:
      nombre_imagen diagnostico_original diagnostico_propuesto  \
0  ISIC_0000000.jpg               benign                benign   
1  ISIC_0000001.jpg               benign                benign   
2  ISIC_0000003.jpg               benign                benign   
3  ISIC_0000006.jpg               benign                benign   
4  ISIC_0000007.jpg               benign                benign   

   confianza_prediccion  
0              0.809904  
1              0.939729  
2              0.908125  
3              0.715470  
4              0.819043  

--- Celda 3 Ejecutada ---
